In [13]:
import os
from pathlib import Path
import pandas as pd

def local_path():
    CME_PATH = os.getenv("CME", r"H:\global data\Ticks for CL&NG&SI\OneMinute\CME")
    CME = Path(CME_PATH)
    return {i.stem: i for i in CME.iterdir()}
SYMBOLS = local_path()
SYMBOLS

{'CL': WindowsPath('H:/global data/Ticks for CL&NG&SI/OneMinute/CME/CL'),
 'GC': WindowsPath('H:/global data/Ticks for CL&NG&SI/OneMinute/CME/GC'),
 'NG': WindowsPath('H:/global data/Ticks for CL&NG&SI/OneMinute/CME/NG'),
 'SI': WindowsPath('H:/global data/Ticks for CL&NG&SI/OneMinute/CME/SI')}

In [51]:
def local_quote(symbol: str, paths: dict[str, Path], symbol_pre: str = "GC"):
    path = paths[symbol_pre]
    contracts = {i.stem[2:] : i for i in path.glob("*.txt")}
    GC_TXT = contracts[symbol]
    df = pd.read_csv(GC_TXT, header=None, parse_dates=["date"], usecols=range(5), names=["date", "local_high", "local_open", "local_low", "local_close"])
    df.date = pd.to_datetime(df.date)
    return df
GC2312 = local_quote("2312", SYMBOLS)
GC2312

,date,local_high,local_open,local_low,local_close
0,2022-02-12 02:39:00,1900.0,1900.0,1900.0,1900.0
1,2022-02-16 01:44:00,1900.0,1900.0,1900.0,1900.0
2,2022-03-03 00:19:00,1963.0,1963.0,1963.0,1963.0
3,2022-03-07 08:23:00,2035.0,2035.0,2035.0,2035.0
4,2022-03-07 10:45:00,2035.0,2035.0,2035.0,2035.0
...,...,...,...,...,...
11154,2023-05-25 16:37:00,2021.4,2021.4,2021.4,2021.4
11155,2023-05-25 16:40:00,2021.4,2021.4,2021.4,2021.4
11156,2023-05-25 16:42:00,2021.0,2021.0,2021.0,2021.0
11157,2023-05-25 16:44:00,2020.4,2020.4,2020.4,2020.4


In [17]:
from ib_insync import *
util.startLoop()
ib = IB()
ib.connect(host="127.0.0.1", port=7496, clientId=12)

<IB connected to 127.0.0.1:7496 clientId=12>

In [68]:
contract = Contract()
contract.localSymbol = "GCZ3"
contract.secType = "FUT"
contract.exchange = "COMEX"

# end_datetime = GC2312["date"].max().strftime("%Y%m%d %H:%M:%S") + " US/Eastern"
end_datetime = "20230523 16:52:00 US/Eastern"
print(end_datetime)
duration = "1 D"
bar_size = "2 mins"
bars = ib.reqHistoricalData(
    contract,
    endDateTime=end_datetime,
    durationStr=duration,
    barSizeSetting=bar_size,
    whatToShow="TRADES",
    useRTH=True,
)
online_GC2312 = util.df(bars)

20230523 16:52:00 US/Eastern


In [64]:
GC2312

,date,local_high,local_open,local_low,local_close
0,2022-02-12 02:39:00,1900.0,1900.0,1900.0,1900.0
1,2022-02-16 01:44:00,1900.0,1900.0,1900.0,1900.0
2,2022-03-03 00:19:00,1963.0,1963.0,1963.0,1963.0
3,2022-03-07 08:23:00,2035.0,2035.0,2035.0,2035.0
4,2022-03-07 10:45:00,2035.0,2035.0,2035.0,2035.0
...,...,...,...,...,...
11154,2023-05-25 16:37:00,2021.4,2021.4,2021.4,2021.4
11155,2023-05-25 16:40:00,2021.4,2021.4,2021.4,2021.4
11156,2023-05-25 16:42:00,2021.0,2021.0,2021.0,2021.0
11157,2023-05-25 16:44:00,2020.4,2020.4,2020.4,2020.4


In [69]:
online_GC2312["date"] = pd.to_datetime(online_GC2312["date"].dt.strftime("%Y%m%d %H:%M:%S"))
online_GC2312

,date,open,high,low,close,volume,average,barCount
0,2023-05-23 09:30:00,2027.1,2027.1,2027.1,2027.1,1.0,2027.10,1
1,2023-05-23 09:32:00,2027.1,2027.1,2027.1,2027.1,0.0,2027.10,0
2,2023-05-23 09:34:00,2024.5,2024.5,2024.5,2024.5,2.0,2024.50,1
3,2023-05-23 09:36:00,2024.5,2024.5,2024.5,2024.5,0.0,2024.50,0
4,2023-05-23 09:38:00,2027.3,2027.5,2027.3,2027.3,21.0,2027.34,14
...,...,...,...,...,...,...,...,...
216,2023-05-23 16:42:00,2032.7,2032.7,2032.7,2032.7,0.0,2032.70,0
217,2023-05-23 16:44:00,2032.7,2032.7,2032.7,2032.7,0.0,2032.70,0
218,2023-05-23 16:46:00,2032.7,2032.7,2032.7,2032.7,0.0,2032.70,0
219,2023-05-23 16:48:00,2032.7,2032.7,2032.7,2032.7,0.0,2032.70,0


In [70]:
compare = pd.merge(GC2312, online_GC2312, on="date", how="inner")
compare.to_csv("GC2312对比_0523.csv")
compare

,date,local_high,local_open,local_low,local_close,open,high,low,close,volume,average,barCount
0,2023-05-23 10:06:00,2018.3,2018.3,2018.3,2018.3,2029.0,2029.0,2029.0,2029.0,0.0,2029.00,0
1,2023-05-23 10:36:00,2019.6,2019.6,2019.6,2019.6,2025.5,2025.5,2025.5,2025.5,1.0,2025.50,1
2,2023-05-23 10:38:00,2020.2,2020.2,2020.2,2020.2,2025.0,2025.0,2024.8,2024.8,21.0,2024.90,3
3,2023-05-23 10:40:00,2020.0,2020.0,2020.0,2020.0,2024.8,2024.8,2024.8,2024.8,0.0,2024.80,0
4,2023-05-23 10:52:00,2021.6,2021.6,2021.6,2021.6,2026.4,2026.4,2026.4,2026.4,1.0,2026.40,1
5,2023-05-23 11:20:00,2018.6,2018.6,2018.6,2018.6,2024.5,2024.5,2024.4,2024.4,10.0,2024.47,10
6,2023-05-23 12:32:00,2020.9,2020.9,2020.9,2020.9,2027.9,2027.9,2027.9,2027.9,1.0,2027.90,1
7,2023-05-23 12:40:00,2022.2,2022.2,2022.2,2022.2,2031.4,2031.4,2031.4,2031.4,0.0,2031.40,0
8,2023-05-23 13:26:00,2020.2,2020.3,2020.3,2020.2,2030.9,2030.9,2030.9,2030.9,2.0,2030.90,1
9,2023-05-23 13:32:00,2018.9,2019.0,2019.1,2018.7,2031.6,2031.6,2031.6,2031.6,0.0,2031.60,0


In [56]:
online_GC2312.date.dtype == GC2312.date.dtype

True